##  The `concurrent.futures` module provides a high-level interface for asynchronously executing callables ( threads or processes).
The asynchronous execution can be performed with 
* With threads, using `ThreadPoolExecutor`, it is used for `I/O bound` operation
* With processes, using `ProcessPoolExecutor`, it is used for `CPU bound` operation
* Easily manage tasks running concurrently and in parallel using both Threads and Process whichever is suitable

Both implement the same interface, which is defined by the abstract `Executor` class.

https://docs.python.org/3/library/concurrent.futures.html

https://www.tutorialspoint.com/concurrency_in_python/concurrency_in_python_pool_of_threads.htm

## 1) Thread Pool Executor
https://docs.python.org/3/library/concurrent.futures.html#threadpoolexecutor

### Subclass `submit` : Schedules the execution of a function on the arguments; takes two input `function` and `arguments` for the function, it returns a future object
* The Future object has a method called `done()`, which tells if the future has resolved

In [7]:
from concurrent.futures import ThreadPoolExecutor
import time

### In the following operation ThreadPoolExecutor has been constructed with 3 threads. Then a task, which will wait for 2 seconds before giving the message, is submitted to the thread pool executor. <b>

In [8]:
def return_after_n_secs(n, message):
    time.sleep(n)
    return message

#### Create a pool of threads with threads 3, default = 5

In [9]:
pool = ThreadPoolExecutor(3) 

#### Submitting a task to the pool of threads

In [10]:
submitted_job = pool.submit(return_after_n_secs, 60, "Hello")

#### Check whether the task finished
If executed less than 60 seconds after the previous cell was run, this will return False

In [11]:
submitted_job.done()

False

#### Access the result of the submitted job
This will wait until the result is available

In [ ]:
print(submitted_job.result())

#### Check the status of the job again
If 60 seconds have elapsed, this will return True

In [ ]:
submitted_job.done()

### Use of `Executor.map()` function
* It applies a certain function to every element within iterables

https://docs.python.org/3/library/concurrent.futures.html#concurrent.futures.Executor.map

In [12]:
num_list = [2, 3, 4, 5, 6]

def cal_square(x):
    return x * x

In [13]:
def executor_func():
    
    with ThreadPoolExecutor(max_workers = 3) as executor:
        results = executor.map(cal_square, num_list)
    
    return results

In [14]:
square_data = executor_func()

square_data

<generator object Executor.map.<locals>.result_iterator at 0x7ff1182f7b40>

In [15]:
list(square_data)

[4, 9, 16, 25, 36]

### Executing the same operation with `submit()` subclass

In [16]:
def executor_func():
    
    with ThreadPoolExecutor(max_workers = 3) as executor:
        results = executor.submit(cal_square, num_list)
        
    return results

#### A TypeError is raised
This is because the submit function does not do a mapping as the map function does. It only calls the function with its arguments

In [17]:
square_data = executor_func()

square_data

<Future at 0x7ff0fef5d850 state=finished raised TypeError>

#### Details of the TypeError

In [18]:
square_data.result()

TypeError: can't multiply sequence by non-int of type 'list'

## 2) Process Pool Executor
https://docs.python.org/3/library/concurrent.futures.html#processpoolexecutor

In [19]:
from concurrent.futures import ProcessPoolExecutor  

### Performing an ` I/O` bound method for

* `Process Pool` and 
* `Thread pool` execution

In computer science, I/O bound refers to a condition in which the time it takes to complete a computation is determined principally by the period spent waiting for input/output operations to be completed

In [20]:
from concurrent.futures import as_completed
import urllib.request

* `concurrent.futures.as_completed()` : Returns an iterator over the Future instances
https://docs.python.org/3/library/concurrent.futures.html#concurrent.futures.as_completed

### Performing  in `ProcessPoolExecutor`

In [21]:
from concurrent.futures import ProcessPoolExecutor

In [22]:
url_list = ['https:wikipedia.org']

In [23]:
def url_loader(url, time):
    with urllib.request.urlopen(url, timeout = time) as conn:
        return conn.read()

In [24]:
def main_processpool():
    start = time.time()
    
    with ProcessPoolExecutor(max_workers = 7) as executor:
        future_to_page = {executor.submit(url_loader, url, 60): url for url in url_list}

        for future in as_completed(future_to_page):
            url = future_to_page[future]
            result = future.result()
            print('The page %r is %d bytes' % (url, len(result)))
            
    print('Total time taken:', time.time() - start)

In [25]:
main_processpool()

URLError: <urlopen error no host given>

### Performing in 'ThreadPoolExecutor'

In [ ]:
def main_threadpool():
    start = time.time()
    
    with ThreadPoolExecutor(max_workers=7) as executor:
        future_to_page = {executor.submit(url_loader, url, 60): url for url in url_list}

        for future in as_completed(future_to_page):
            url = future_to_page[future]
            result = future.result()
            print('The page %r is %d bytes' % (url, len(result)))
            
    print('Total time taken:', time.time() - start)

In [ ]:
main_threadpool()

<b> `output`: We can see that the `Thread pool execution` gives better execution time

### Performing a CPU bound method for
* `Sequential`, 
* `Thread Pool` and 
* `Process pool` execution

In [ ]:
num_list = [1, 2, 3, 4, 5, 6]

In [ ]:
def count(number):
    for i in range(0, 10000000):
        i=i+1
    return i**number        

In [ ]:
def asses_item(x):
    result_item = count(x)
    print("item " + str(x) + " result " + str(result_item))

In [ ]:
start_time = time.time()

for item in num_list:
    asses_item(item)
    
print("Sequential execution in " + str(time.time() - start_time), "seconds")

In [ ]:
start_time = time.time()

with ThreadPoolExecutor(max_workers=4) as executor:
    for item in num_list:
        executor.submit(asses_item, item)
        
print("Thread pool execution in " + str(time.time() - start_time), "seconds")    

 <b> `output`: In the output there may be two results printed in interleaved fashion, which can be resolved using locks, we have seen already, which is not our priority right now, our concern is just to show the time consumption of Thread pool execution to compare with others </b>

In [ ]:
start_time = time.time()

with ProcessPoolExecutor(max_workers=4) as executor:
    for item in num_list:
        executor.submit(asses_item, item) 
        
print("Thread pool execution in " + str(time.time() - start_time), "seconds") 

<b> `output`: For `CPU bound` the best performance is given by `ProcessPoolExecutor`